<a href="https://colab.research.google.com/github/Muyiiwaa/machine_learning_notes/blob/master/Transfer_completed_Learning_in_Computer_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn, optim
import numpy as np
import timm
import wandb
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import kagglehub
import os
from tqdm import tqdm

In [ ]:
# Download latest version
path = kagglehub.dataset_download("alessiocorrado99/animals10")

print("Path to dataset files:", path)
data_path = os.listdir(path)
data_path = os.path.join(path, data_path[1])
data_path

Path to dataset files: /kaggle/input/animals10


'/kaggle/input/animals10/raw-img'

In [ ]:
## load and prepare the dataset

# setup the transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip(p=0.4),
    transforms.ColorJitter(brightness=0.2, saturation=0.2,
                           hue=0.1,contrast=0.1),
    transforms.Resize((200,200))
])

# load the images
data = datasets.ImageFolder(root=data_path,transform=transform)


# split the data into train and test set
train_size = int(0.6 * len(data))
test_size = len(data) - train_size
train_data, test_data = random_split(dataset=data, lengths=[train_size, test_size])

# repeat this step to split test (40% of the data) into train and test set to fit gpu.
train_size = int(0.8 * len(test_data))
test_size = len(test_data) - train_size
train_data, test_data = random_split(dataset=test_data, lengths=[train_size, test_size])


# setup the data loader
BATCH_SIZE = 32
train_loader = DataLoader(dataset = train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# setup the model architecture and hyperparameters

model = timm.create_model(model_name='efficientnet_b3', pretrained=True)
model.classifier = nn.Linear(in_features=model.classifier.in_features, out_features=10)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
LEARNING_RATE = 1e-4
optimizer = optim.AdamW(params=model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=2, gamma=0.1)
EPOCHS = 10
criterion = nn.CrossEntropyLoss()

In [ ]:
animal_list = ['dog','horse','elephant',
               'butterfly','chicken','cat',
               'cow','sheep','spider','squirrel']
len(animal_list)

10

In [ ]:
# init wandb for tracking

run = wandb.init(
    project = 'Animal Classification With Fine Tuned EfficientNet-B3',
    name= 'second run',
    config= {
        'model_name': 'efficientnet-b3',
        'device': device,
        'data_path': data_path,
        'learning rate': LEARNING_RATE,
        'optimizer': 'AdamW',
        'weight decay': 0.01,
        'scheduler': {'step': 2, 'gamma': 0.1},
        'epochs': EPOCHS
    }
)

In [ ]:
# begin the training loop


for epoch in range(EPOCHS):
  train_epoch_loss, test_epoch_loss = [], []
  train_epoch_labels, train_epoch_preds = [], []
  test_epoch_labels, test_epoch_preds = [], []
  train_batch = tqdm(train_loader, desc= f'Training Epoch: {epoch+1}/{EPOCHS}')
  for image, label in train_batch:
    model.train()
    image, label = image.to(device), label.to(device)
    train_preds = model(image)
    loss = criterion(train_preds, label)

    # back propagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_epoch_loss.append(loss.item())
    train_epoch_labels.extend(label.cpu().detach().numpy())
    _, train_preds = torch.max(train_preds, 1)
    train_epoch_preds.extend(train_preds.cpu().detach().numpy())

    train_batch.set_postfix(loss=loss.item())

  scheduler.step()
  # compute the f1_score, precision and recall for training set
  train_final_loss = sum(train_epoch_loss)/len(train_epoch_loss)
  train_f1 = f1_score(train_epoch_labels, train_epoch_preds, average='weighted')
  train_precision = precision_score(train_epoch_labels, train_epoch_preds, average='weighted')
  train_recall = recall_score(train_epoch_labels, train_epoch_preds, average='weighted')

  # now evaluate
  model.eval()
  with torch.no_grad():
    test_batch = tqdm(test_loader, desc= f'Evaluating tests for Epoch: {epoch+1}/{EPOCHS}')
    for test_image, test_labels in test_batch:
      test_image, test_labels = test_image.to(device), test_labels.to(device)
      test_preds = model(test_image)


      test_loss = criterion(test_preds, test_labels)
      test_epoch_loss.append(test_loss.item())
      test_epoch_labels.extend(test_labels.cpu().detach().numpy())
      _, test_preds = torch.max(test_preds, 1)
      test_epoch_preds.extend(test_preds.cpu().detach().numpy())

  # compute the f1_score, precision and recall for test set
  test_final_loss = sum(test_epoch_loss)/len(test_epoch_loss)
  test_f1 = f1_score(test_epoch_labels, test_epoch_preds, average='weighted')
  test_precision = precision_score(test_epoch_labels, test_epoch_preds, average='weighted')
  test_recall = recall_score(test_epoch_labels, test_epoch_preds, average='weighted')

  print(f'Completed training Epoch: {epoch+1}..f1: {train_f1, test_f1}, loss: {train_final_loss, test_final_loss}')
  print(f'Completed training Epoch: {epoch+1}..recall: {train_recall, test_recall}, precision: {train_precision, test_precision}')

  # append result in wandb
  run.log({
      'epochs': epoch + 1,
      'train loss': train_final_loss,
      'test loss': test_final_loss,
      'train f1': train_f1,
      'train precision': train_precision,
      'train recall': train_recall,
      'test f1': test_f1,
      'test precision': test_precision,
      'test recall': test_recall
  })

run.finish()

Evaluating tests for Epoch: 1/10: 100%|██████████| 66/66 [01:05<00:00,  1.01it/s]


Completed training Epoch: 1..f1: (0.8064405237682241, 0.9421279082080406), loss: (0.7354383390567457, 0.20756088710869802)
Completed training Epoch: 1..recall: (0.8082845887549241, 0.9422434367541767), precision: (0.815179734088142, 0.9430749579213973)


Evaluating tests for Epoch: 2/10: 100%|██████████| 66/66 [01:04<00:00,  1.03it/s]


Completed training Epoch: 2..f1: (0.9435688350336278, 0.9483890397095958), loss: (0.18579392459684316, 0.17063757940901048)
Completed training Epoch: 2..recall: (0.9436552465082966, 0.9484486873508353), precision: (0.9436160798323323, 0.9487741604875984)


Evaluating tests for Epoch: 3/10: 100%|██████████| 66/66 [01:04<00:00,  1.02it/s]


...

Evaluating tests for Epoch: 9/10: 100%|██████████| 66/66 [01:04<00:00,  1.03it/s]


Completed training Epoch: 9..f1: (0.9848300770481642, 0.9527316915998815), loss: (0.06391019183779775, 0.16265693013415192)
Completed training Epoch: 9..recall: (0.9848394413274442, 0.952744630071599), precision: (0.9848504506758248, 0.9529854897330488)


Evaluating tests for Epoch: 10/10: 100%|██████████| 66/66 [01:04<00:00,  1.02it/s]

Completed training Epoch: 10..f1: (0.985185950888213, 0.950168494200134), loss: (0.06393815954713478, 0.163572223363162)
Completed training Epoch: 10..recall: (0.9851975647606541, 0.9503579952267303), precision: (0.9851957383169929, 0.9504840315245299)


epochs,▁▂▃▃▄▅▆▆▇█
test f1,▁▄▇██▆▆█▆▅
test loss,█▄▁▂▁▂▂▁▃▃
test precision,▁▄▆█▇▇▆█▆▄
test recall,▁▄▇██▇▆█▆▅
train f1,▁▆████████
train loss,█▂▁▁▁▁▁▁▁▁
train precision,▁▆████████
train recall,▁▆████████
epochs,10
test f1,0.95017


In [ ]:
from PIL import Image

softmax = nn.Softmax()


def predict_animal() -> None:
  new_image = Image.open(input('Enter image url: ')).convert('RGB')
  new_image = transform(new_image).unsqueeze(0)
  new_image = new_image.to(device)
  preds = model(new_image)
  preds = softmax(preds)
  probability, prediction = torch.max(preds, 1)

  print(f'This is an image of {animal_list[prediction.item()]} with probability of {probability.item()}')




predict_animal()


Enter image url: /content/some_image.jpeg
This is an image of dog with probability of 0.9983893632888794
